<a href="https://colab.research.google.com/github/juliablaz2003/UFV_visualizacion/blob/main/Ejercicios%20de%20clase/clase2_18oct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nº de filas

Nº Actuals y forecasts

Horizonte de predicción

Nº de paises y de productos

Historico de actuals/forecast

Cuantas ejecuciones de forecast hay (tener en cuenta el año también)

Forecast distintos

In [4]:
import pandas as pd

# Enlace al archivo CSV en la versión "raw" de GitHub
url = '/content/datos_ejercicio_ventas.csv'

# Leer el archivo CSV directamente desde GitHub
df = pd.read_csv(url, sep = ';')

# Mostrar las primeras 5 filas del DataFrame
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,"7,54356E+15"
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,"5,60031E+15"
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15"
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,"3,63225E+15"
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,"3,96176E+16"


In [5]:
print(df.columns)

Index(['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH', 'SCENARIO', 'FORECAST',
       'FORECAST_YEAR', 'AMOUNT'],
      dtype='object')


In [6]:
print(df.dtypes)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT            object
dtype: object


In [7]:
# 1. Número total de filas en el archivo
num_filas = df.shape[0]
num_filas

18666

In [8]:
# Número de actuals (usando YEAR y MONTH)
num_actuals = df[(df['SCENARIO'] == 'actual') & (df['YEAR'].notnull()) & (df['MONTH'].notnull())].shape[0]

# Número de forecasts (usando FORECAST y FORECAST_YEAR)
num_forecasts = df[(df['SCENARIO'] == 'AI_forecast') & (df['FORECAST_YEAR'].notnull())].shape[0]

print(f"Número de actuals: {num_actuals}")
print(f"Número de forecasts: {num_forecasts}")


Número de actuals: 900
Número de forecasts: 17766


In [9]:
# 3. Horizonte de predicción
df['Prediction_Horizon'] = df['YEAR'] - df['FORECAST_YEAR']

In [10]:
# 4. Número de países y productos
num_paises = df['COUNTRY'].nunique()
num_productos = df['SUBBRAND'].nunique()

num_paises, num_productos

(9, 6)

In [11]:
# 5. Histórico de actuals/forecast
historico_forecast = df[df['SCENARIO'] == 'AI_forecast']
historico_forecast

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Prediction_Horizon
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,"7,54356E+15",0.0
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,"5,60031E+15",0.0
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15",0.0
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,"3,63225E+15",1.0
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,"3,96176E+16",0.0
...,...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,"1,31351E+16",1.0
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,"1,3144E+16",1.0
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0,1.0
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,"5,33063E+15",1.0


In [12]:
# 6. Cuántas ejecuciones de forecast hay (tener en cuenta el año también)
num_ejecuciones = df[['FORECAST', 'YEAR', 'COUNTRY']].drop_duplicates().shape[0]
num_ejecuciones

279

In [13]:
# 7. Forecast distintos
num_forecasts_distintos = df['FORECAST'].nunique()
num_forecasts_distintos

12

In [14]:
# 8. Distribución de ventas
# Distribución por país
ventas_pais = df.groupby('COUNTRY')['AMOUNT'].sum()

# Distribución por mes y año
ventas_mes_anio = df.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum()

# Distribución por marca
ventas_marca = df.groupby('SUBBRAND')['AMOUNT'].sum()

In [16]:
# Distribución de ventas por país
ventas_pais = df.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
ventas_pais.columns = ['País', 'Ventas Totales']
ventas_pais

,País,Ventas Totales
0,Czech,"6,40546E+153,70161E+153,17424E+151,04041E+163,..."
1,Denmark,"03,18214E+151,50029E+152,05305E+165,05772E+161..."
2,Great Britain,"5,60031E+153,63225E+151,13856E+232,13197E+164,..."
3,Hungary,"3,96176E+167,74392E+152,95794E+154,35904E+1502..."
4,Italy,"2,07474E+161,86723E+162,52571E+151,5507E+161,0..."
5,Netherlands,"005,96039E+145,53577E+158,75614E+151,13841E+15..."
6,Norway,"2,66727E+162,60724E+162,84449E+142,43836E+168,..."
7,Portugal,"7,54356E+151,70947E+164,27355E+161,833E+162,29..."
8,Spain,"8,8502E+151,34268E+159,70222E+157,01443E+153,8..."


In [ ]:
# 9. Tendencia y estacionalidad
# País con menos ventas
pais_menos_ventas = ventas_pais.idxmin()
tendencia_pais_menos_ventas = df[df['COUNTRY'] == pais_menos_ventas].groupby('YEAR')['AMOUNT'].sum()

# Marca con más ventas
marca_mas_ventas = ventas_marca.idxmax()
tendencia_marca_mas_ventas = df[df['SUBBRAND'] == marca_mas_ventas].groupby('YEAR')['AMOUNT'].sum()

tendencia_pais_menos_ventas, tendencia_marca_mas_ventas

(YEAR
 2023    05,53577E+158,75614E+1505,86842E+156,8071E+151...
 2024    05,96039E+141,13841E+154,26484E+165,18316E+161...
 2025    008,77932E+151,01321E+169,10517E+153,91441E+15...
 Name: AMOUNT, dtype: object,
 YEAR
 2023    8,8502E+151,13856E+238,75614E+153,17424E+152,2...
 2024    2,60724E+169,64887E+151,13841E+155,95767E+133,...
 2025    3,18214E+152,07904E+157,89979E+158,02372E+154,...
 Name: AMOUNT, dtype: object)

In [ ]:
# 10. Predicciones hechas en España
predicciones_espana = df[df['COUNTRY'] == 'Spain']
predicciones_espana

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Prediction_Horizon
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15",0.0
20,Spain,Pepsi Regular (L3),2023,12,AI_forecast,AI_P05F,2023.0,"1,34268E+15",0.0
25,Spain,Lipton (L3),2025,3,AI_forecast,AI_P11F,2023.0,"9,70222E+15",2.0
62,Spain,7up Free (L3),2024,1,AI_forecast,AI_P04F,2023.0,"7,01443E+15",1.0
68,Spain,7up (L3),2024,4,AI_forecast,AI_P02F,2023.0,"3,88829E+15",1.0
...,...,...,...,...,...,...,...,...,...
18599,Spain,7up Free (L3),2023,9,actual,NaN,NaN,"7,58888E+14",NaN
18611,Spain,Pepsi Regular (L3),2024,1,AI_forecast,AI_P07F,2023.0,"1,0708E+16",1.0
18617,Spain,Pepsi Max (L3),2024,10,AI_forecast,AI_P07F,2023.0,"9,67285E+15",1.0
18653,Spain,Pepsi Regular (L3),2024,3,AI_forecast,AI_P09F,2023.0,"1,20653E+16",1.0
